In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

In [2]:
url = 'https://mydramalist.com/shows/top?page=1'


the problem is after clicking back, title_elements is no longer valid
also titles only exist on the even indices

In [38]:
# THIS CODE WILL LOOP THROUGH ALL 20 TITLE ELEMENTS AND GRAB THE GENRES AND PUT THEM INTO A LIST!!!!!
genres = []
def initialize_browser():
    ## create the driver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.set_page_load_timeout(10)
    
    ## get the url
    try:
        driver.get(url)
    except TimeoutException as error:
        print("timeout error- getting url")   
    #print('Starting Driver')

    ## find all the show titles
    title_elements = driver.find_elements(by=By.CSS_SELECTOR, value='h6.title a')

    # iterate through shows to scrape info
    for index in range(0, len(title_elements), 2):
        ## re-find the element in each iteration
        title_elements = driver.find_elements(By.CSS_SELECTOR, 'h6.title a')
        element = title_elements[index]   
        print(element.text)

        ## scroll to the title, then click
        print("Clicking on the title...")
        try:
            driver.execute_script("arguments[0].scrollIntoView();", element)
            driver.execute_script("window.scrollBy(0, arguments[0]);", -100)
            time.sleep(1)
            element.click()
        except TimeoutException as error:
            print("Timeout error- clicking title ")   
        
        # scrape the genre
        print("Searching for genre...")
        try:
            genres_section = driver.find_element(By.CLASS_NAME, 'show-genres')
            print(genres_section.text)        
            genres.append(genres_section.text)
        except:
            print("Genre not found")

        # go back to title list
        print("going back to main...")
        try:
            driver.execute_script("window.history.go(-1)")
        except TimeoutException as error:
            print("timeout error - going back")
    
initialize_browser()

timeout error- getting url
Twinkling Watermelon
Clicking on the title...
Timeout error- clicking title 
Searching for genre...
Genres: Romance, Youth, Drama, Fantasy
going back to main...
timeout error - going back
Move to Heaven
Clicking on the title...
Timeout error- clicking title 
Searching for genre...
Genres: Life, Drama
going back to main...
Moving
Clicking on the title...
Timeout error- clicking title 
Searching for genre...
Genres: Action, Thriller, Mystery, Supernatural
going back to main...
timeout error - going back
Weak Hero Class 1
Clicking on the title...
Timeout error- clicking title 
Searching for genre...
Genres: Action, Youth, Drama
going back to main...
timeout error - going back
Hospital Playlist Season 2
Clicking on the title...
Timeout error- clicking title 
Searching for genre...
Genres: Romance, Life, Drama, Medical
going back to main...
timeout error - going back
Nirvana in Fire
Clicking on the title...
Timeout error- clicking title 
Searching for genre...
Gen